In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle as pkl

In [3]:
os.listdir("data/")
# os.listdir("data/peptides_train_to_mic.csv")


['peptides_train_to_mic.csv',
 'peptides_w_mic_train.txt',
 'peptides_2024_cdhit90_unbalanced_test.txt',
 'peptides_predicted-log10mic_zScoreNormalized_train.txt',
 'vocab',
 'single_point_initialization.pkl',
 'peptides_w_mic_test.txt',
 'properties',
 'peptides_2024_cdhit90_unbalanced_train.txt',
 'peptides_predicted-log10mic_zScoreNormalized_test.txt',
 'peptides_predicted_log10_mic_test.txt',
 'ten_point_initialization.pkl',
 'top10_predict_mic_features.pkl',
 'peptides_2024_cdhit90_unbalanced_train_function.txt',
 'peptides_predicted_log10_mic_train.txt',
 'peptides_2024_cdhit90_unbalanced_test_function.txt']

In [4]:
seq2prop_fpath = 'data/peptides_train_to_mic.csv'

In [5]:
N_INITALIZATION_POINTS  = 100
N_DIFFERENT_INITIALIZATIONS = 5

In [6]:
train_seqs_to_prop = pd.read_csv(seq2prop_fpath)

sampled_seqs_to_prop = train_seqs_to_prop.sample(N_INITALIZATION_POINTS*N_DIFFERENT_INITIALIZATIONS, 
                                                 ignore_index=True, 
                                                 random_state=42
)

In [7]:
sampled_seqs_to_prop.shape

(500, 3)

In [8]:
sampled_seqs_to_prop.head()

,sequence,MIC,log10_mic
0,SRWPSPGRPRPFPGRPKPIFRPRPCNCYAPPCPCDRW,11.150000,1.047275
1,HVINLEESFQEPEYENHLA,0.600000,-0.221849
2,GLMDVFKGAAKNLLASALDKIRCKVTKC,10.666667,1.028029
3,DNGYLLDKYTGCKVWCVINNESCNSECKIRGGYYGYCYFWKLACFC...,4.500000,0.653213
4,ALYKKFKKKLLKSLKRLG,2.311873,0.363964


In [9]:
run0_init = sampled_seqs_to_prop.iloc[:100]
run1_init = sampled_seqs_to_prop.iloc[100:200]
run2_init = sampled_seqs_to_prop.iloc[200:300]
run3_init = sampled_seqs_to_prop.iloc[300:400]
run4_init = sampled_seqs_to_prop.iloc[400:]

In [10]:
runs_init = [
    run0_init,
    run1_init,
    run2_init,
    run3_init,
    run4_init
]

In [11]:
for i in range(5):
    runs_init[i].reset_index(drop=True, inplace=True)

In [12]:
for i in range(5):
    print( runs_init[i].log10_mic.min() )

-0.7099653886374819
-0.7042050491301658
-0.551112314497439
-0.721246399047171
-0.3193281190869733


In [13]:
# now make three more datasets 
single_point_data_set = {}
each_run_min_val_idx = {}
for _run_ix in range(5):
    for i,v in enumerate(runs_init[_run_ix].log10_mic):
        if v == np.min(runs_init[_run_ix].log10_mic):
            single_point_data_set[f"run_{_run_ix}"] = runs_init[_run_ix].iloc[i]
            each_run_min_val_idx[f"run_{_run_ix}"] = i

In [14]:
runs_init[1].head()

,sequence,MIC,log10_mic
0,GRFKRFRKKFKKLFKKASPVIPLLHL,5.100000,0.707570
1,RKCLRWQWRMRKYGG,10.000000,1.000000
2,GLLSRLRDFLSDRGRRLGEKIERIGQKIKDLSEFFQS,1.600000,0.204120
3,PKRKAEGDAKGDKAKVKDEPQRRSARLSAKPAPPKPEPKPKKAPAK...,1.349694,0.130235
4,KLLKQWPIGKLLKKLLKKLLK,0.197604,-0.704205


In [15]:
each_run_min_val_idx

{'run_0': 63, 'run_1': 4, 'run_2': 93, 'run_3': 21, 'run_4': 45}

In [16]:
mask_per_run = {}
for i in range(5):
    mask_per_run[f"run_{i}"] = np.array([False]*100)
    mask_per_run[f"run_{i}"][ each_run_min_val_idx[f"run_{i}"] ] = True

In [68]:
for i in range(5):
    _idx_of_min = each_run_min_val_idx[f"run_{i}"]
    _sampled_indices = np.random.choice(list(range(_idx_of_min)) + list(range(_idx_of_min+1, 100)), replace=False, size=9)
    print(_sampled_indices, _idx_of_min)
    if (_sampled_indices==_idx_of_min).sum()>0:
        print("duplicate index with idx of min")

    mask_per_run[f"run_{i}"][ _sampled_indices ] = True

[31 23 75 40 91 62 30  2 95] 63
[22 51  2 14 82 69 62 23 29] 4
[28 27 52 75  4 79 32 56 50] 93
[ 5 26  4 39 29 77 90 45 70] 21
[94 32 19  7 78 67 50 58 52] 45


In [77]:
ten_point_data_set = {}
for i in range(5):
    _k = f"run_{i}"
    ten_point_data_set[_k] = runs_init[i].iloc[mask_per_run[_k]]
    ten_point_data_set[_k].reset_index(drop=True, inplace=True)

In [78]:
ten_point_data_set[_k]

,sequence,MIC,log10_mic
0,GFFALIPKIISSPLFKTL,17.064461,1.232093
1,AIGSILGALAKGLPTLKSWIKNR,25.000000,1.397940
2,KFWSLLKKALRLWANVL,5.751076,0.759749
3,KWKSFLKTFKSAVKTVLHTALKAISS,0.479371,-0.319328
4,KRIVKRIKKWLR,2.000000,0.301030
5,FPVTWRWWTWWKG,16.714113,1.223083
6,FASGIAGMAGKLF,200.000000,2.301030
7,RCRCRCRCRC,15.213079,1.182217
8,FSKYERQKDKRPYSERKNQYTGPQFLYPPERIPPQKVIKWNEEGLP...,1.659518,0.219982
9,GVVVRVGRVVVRWVRRRR,4.000000,0.602060


In [80]:
single_point_data_set[_k]

sequence     KWKSFLKTFKSAVKTVLHTALKAISS
MIC                            0.479371
log10_mic                     -0.319328
Name: 45, dtype: object

In [85]:
with open("data/single_point_initialization.pkl", "wb") as f:
    pkl.dump(single_point_data_set, f)

In [86]:
with open("data/ten_point_initialization.pkl", "wb") as f:
    pkl.dump(ten_point_data_set, f)

In [87]:
os.listdir("data")

['peptides_train_to_mic.csv',
 'peptides_w_mic_train.txt',
 'peptides_2024_cdhit90_unbalanced_test.txt',
 'peptides_predicted-log10mic_zScoreNormalized_train.txt',
 'vocab',
 'single_point_initialization.pkl',
 'peptides_w_mic_test.txt',
 'properties',
 'peptides_2024_cdhit90_unbalanced_train.txt',
 'peptides_predicted-log10mic_zScoreNormalized_test.txt',
 'peptides_predicted_log10_mic_test.txt',
 'ten_point_initialization.pkl',
 'top10_predict_mic_features.pkl',
 'peptides_2024_cdhit90_unbalanced_train_function.txt',
 'peptides_predicted_log10_mic_train.txt',
 'peptides_2024_cdhit90_unbalanced_test_function.txt']

In [106]:
_temp_dframes = []
for i in range(5):
    _k = f"run_{i}"
    _temp_dframes.append( 
        pd.DataFrame({
            "sequence" :[single_point_data_set[_k].sequence],
            "MIC"      :[single_point_data_set[_k].MIC],
            "log10_mic":[single_point_data_set[_k].log10_mic]
        })
    )


In [107]:
tst = pd.concat(_temp_dframes)

In [108]:
tst.shape

(5, 3)

In [109]:
tst

,sequence,MIC,log10_mic
0,MGGLKKLGKKLEGAGKRVFNAAEKALPIAQQAANVAATARG,0.195000,-0.709965
0,KLLKQWPIGKLLKKLLKKLLK,0.197604,-0.704205
0,RRWCFRVCYRGFCYRKCR,0.281117,-0.551112
0,AKKVFKRLPKLFSKIWNWK,0.190000,-0.721246
0,KWKSFLKTFKSAVKTVLHTALKAISS,0.479371,-0.319328


In [17]:
with open("data/ten_point_initialization.pkl", "rb") as f:
    ten_point_data_set = pkl.load(f)

# make initial sets of 20 points

In [ ]:
twenty_point_data_set = {}

In [28]:
## indices that already exist in the ten_point_data_set
# we want to sample another ten points that are not these
run0_ixs_contained = [31, 23, 75, 40, 91, 62, 30,  2, 95, 63]
run1_ixs_contained = [22, 51,  2, 14, 82, 69, 62, 23, 29, 4]
run2_ixs_contained = [28, 27, 52, 75,  4, 79, 32, 56, 50, 93]
run3_ixs_contained = [ 5, 26,  4, 39, 29, 77, 90, 45, 70, 21]
run4_ixs_contained = [94, 32, 19,  7, 78, 67, 50, 58, 52, 45]
runs_ixs_contained = [
    run0_ixs_contained,
    run1_ixs_contained,
    run2_ixs_contained,
    run3_ixs_contained,
    run4_ixs_contained
]

In [23]:
tst = list(range(8))
print(tst)
tst.remove(2)
tst.remove(4)
tst.remove(6)
print(tst)

[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 3, 5, 7]


In [29]:
for i in range(5):
    mask_per_run[f"run_{i}"] = np.array([False]*100)

for i in range(5): # each run
    _bag_of_ixs = list(range(100))

    # remove indices we already have from the bag
    for j in range(10): # over each already existing ix array
        _bag_of_ixs.remove( runs_ixs_contained[i][j] ) # don't sample indices we already have

    # sample from the bag
    _sampled_indices = np.random.choice(_bag_of_ixs, replace=False, size=10)
    
    # check for dupes
    if len(set(list(_sampled_indices) + runs_ixs_contained[i]) ) < 20:
        print("duplicate indices")
    
    # add to contained collection
    runs_ixs_contained[i] = runs_ixs_contained[i] + list(_sampled_indices)
        
    _sampled_indices = np.array( runs_ixs_contained[i] )
    mask_per_run[f"run_{i}"][ _sampled_indices ] = True
    print("count =",mask_per_run[f"run_{i}"].sum())



count = 20
count = 20
count = 20
count = 20
count = 20


In [30]:
runs_ixs_contained[0]

[31, 23, 75, 40, 91, 62, 30, 2, 95, 63, 99, 72, 36, 33, 49, 13, 83, 32, 55, 77]

In [31]:
each_run_min_val_idx

{'run_0': 63, 'run_1': 4, 'run_2': 93, 'run_3': 21, 'run_4': 45}

In [32]:
twenty_point_data_set = {}
for i in range(5):
    _k = f"run_{i}"
    twenty_point_data_set[_k] = runs_init[i].iloc[mask_per_run[_k]]
    twenty_point_data_set[_k].reset_index(drop=True, inplace=True)
    print(twenty_point_data_set[_k].shape)

(20, 3)
(20, 3)
(20, 3)
(20, 3)
(20, 3)


In [33]:
with open("data/twenty_point_initialization.pkl", "wb") as f:
    pkl.dump(twenty_point_data_set, f)

# again but 50 point dataset

In [34]:
amount_to_add = 30
total_amount = 50
print(f"total amount will be: {amount_to_add}+{len(runs_ixs_contained[0])} = {amount_to_add+len(runs_ixs_contained[0])}")

for i in range(5):
    mask_per_run[f"run_{i}"] = np.array([False]*100)

for i in range(5): # each run
    _bag_of_ixs = list(range(100))

    # remove indices we already have from the bag
    for j in range( len(runs_ixs_contained[i]) ): # over each already existing ix array
        _bag_of_ixs.remove( runs_ixs_contained[i][j] ) # don't sample indices we already have

    # sample from the bag
    _sampled_indices = np.random.choice(_bag_of_ixs, replace=False, size=amount_to_add)
    
    # check for dupes
    if len(set(list(_sampled_indices) + runs_ixs_contained[i]) ) < total_amount:
        print("duplicate indices")
    
    # add to contained collection
    runs_ixs_contained[i] = runs_ixs_contained[i] + list(_sampled_indices)
        
    _sampled_indices = np.array( runs_ixs_contained[i] )
    mask_per_run[f"run_{i}"][ _sampled_indices ] = True
    print("count =",mask_per_run[f"run_{i}"].sum())



total amount will be: 30+20 = 50
count = 50
count = 50
count = 50
count = 50
count = 50


In [36]:
fifty_point_data_set = {}
for i in range(5):
    _k = f"run_{i}"
    fifty_point_data_set[_k] = runs_init[i].iloc[mask_per_run[_k]]
    fifty_point_data_set[_k].reset_index(drop=True, inplace=True)
    print(fifty_point_data_set[_k].shape)

(50, 3)
(50, 3)
(50, 3)
(50, 3)
(50, 3)


In [37]:
with open("data/fifty_point_initialization.pkl", "wb") as f:
    pkl.dump(fifty_point_data_set, f)